# Great Plains towns on abandoned railroad lines

### Cleaning town data from QGIS

I collected places from QGIS in two rounds: NE, SD, and NE; and KS, OK, TX, and MT.

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

In [61]:
places_qgis_1 = pd.read_csv("output-data/places-on-abandoned-selected-NE-SD-NE.csv")
places_qgis_1

,GISJOIN,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area
0,G31000415,31,415,2393893,3100415,Ainsworth,Ainsworth city,25,C5,N,N,G4110,A,2600318.0,0.0,42.548439,-99.857522,10070.192196,2.600319e+06
1,G31000555,31,555,2393908,3100555,Albion,Albion city,25,C1,N,N,G4110,A,2684228.0,0.0,41.687298,-97.998717,13106.718964,2.684228e+06
2,G31001290,31,1290,2397957,3101290,Amherst,Amherst village,47,C1,N,N,G4110,A,565573.0,0.0,40.838372,-99.269509,3496.559126,5.655728e+05
3,G31001465,31,1465,2397965,3101465,Anoka,Anoka village,47,C1,N,N,G4110,A,1451109.0,0.0,42.946000,-98.828529,4834.383737,1.451108e+06
4,G31001850,31,1850,2397977,3101850,Arcadia,Arcadia village,47,C1,N,N,G4110,A,1479286.0,0.0,41.424823,-99.126062,5967.544623,1.479289e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,G46070740,46,70740,1267650,4670740,Wetonka,Wetonka town,43,C5,N,N,G4110,A,637996.0,0.0,45.624216,-98.771741,3196.768382,6.379948e+05
324,G46070940,46,70940,1267651,4670940,White,White city,25,C5,N,N,G4110,A,2262343.0,0.0,44.435904,-96.646010,11558.561770,2.262345e+06
325,G46072100,46,72100,2628854,4672100,Winfred,Winfred CDP,57,U1,N,N,G4210,S,2591544.0,98921.0,43.998795,-97.361614,6556.235129,2.690463e+06
326,G46072180,46,72180,1267659,4672180,Winner,Winner city,25,C5,N,N,G4110,A,5950193.0,0.0,43.377616,-99.855133,27113.181384,5.950197e+06


In [62]:
places_qgis_2 = pd.read_csv("output-data/places-on-abandoned-selected-KS-OK-TX-MT.csv")
places_qgis_2

,GISJOIN,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area
0,G20000225,20,225,2583496,2000225,Ada,Ada CDP,57,U1,N,N,G4210,S,6580260.0,0.0,39.162200,-97.885948,11486.501445,6.580262e+06
1,G20000325,20,325,2393883,2000325,Admire,Admire city,25,C1,N,N,G4110,A,836798.0,0.0,38.640995,-96.102009,3669.722832,8.367972e+05
2,G20000475,20,475,2393890,2000475,Agenda,Agenda city,25,C1,N,N,G4110,A,448549.0,0.0,39.706567,-97.432239,3738.581105,4.485498e+05
3,G20001275,20,1275,2393921,2001275,Allen,Allen city,25,C1,N,N,G4110,A,715808.0,0.0,38.655122,-96.169635,3563.011794,7.158088e+05
4,G20001350,20,1350,2393926,2001350,Alma,Alma city,25,C1,N,N,G4110,A,1514197.0,37583.0,39.015681,-96.287096,9656.745038,1.551786e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,G48075188,48,75188,2412147,4875188,Vega,Vega city,25,C1,N,N,G4110,A,3466375.0,0.0,35.245302,-102.429247,12874.835121,3.466376e+06
432,G48077044,48,77044,2412203,4877044,Weinert,Weinert city,25,C1,N,N,G4110,A,1251893.0,0.0,33.323400,-99.673674,4520.349585,1.251895e+06
433,G48077152,48,77152,2412207,4877152,Wellington,Wellington city,25,C1,N,N,G4110,A,3572564.0,0.0,34.853813,-100.213982,12453.900394,3.572563e+06
434,G48079180,48,79180,2805826,4879180,Wildorado,Wildorado CDP,57,U1,N,N,G4210,S,6090285.0,890.0,35.209206,-102.209515,13205.348811,6.091175e+06


In [63]:
places_qgis = pd.concat([places_qgis_1, places_qgis_2])
places_qgis

,GISJOIN,STATEFP,PLACEFP,PLACENS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,PCICBSA,PCINECTA,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area
0,G31000415,31,415,2393893,3100415,Ainsworth,Ainsworth city,25,C5,N,N,G4110,A,2600318.0,0.0,42.548439,-99.857522,10070.192196,2.600319e+06
1,G31000555,31,555,2393908,3100555,Albion,Albion city,25,C1,N,N,G4110,A,2684228.0,0.0,41.687298,-97.998717,13106.718964,2.684228e+06
2,G31001290,31,1290,2397957,3101290,Amherst,Amherst village,47,C1,N,N,G4110,A,565573.0,0.0,40.838372,-99.269509,3496.559126,5.655728e+05
3,G31001465,31,1465,2397965,3101465,Anoka,Anoka village,47,C1,N,N,G4110,A,1451109.0,0.0,42.946000,-98.828529,4834.383737,1.451108e+06
4,G31001850,31,1850,2397977,3101850,Arcadia,Arcadia village,47,C1,N,N,G4110,A,1479286.0,0.0,41.424823,-99.126062,5967.544623,1.479289e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,G48075188,48,75188,2412147,4875188,Vega,Vega city,25,C1,N,N,G4110,A,3466375.0,0.0,35.245302,-102.429247,12874.835121,3.466376e+06
432,G48077044,48,77044,2412203,4877044,Weinert,Weinert city,25,C1,N,N,G4110,A,1251893.0,0.0,33.323400,-99.673674,4520.349585,1.251895e+06
433,G48077152,48,77152,2412207,4877152,Wellington,Wellington city,25,C1,N,N,G4110,A,3572564.0,0.0,34.853813,-100.213982,12453.900394,3.572563e+06
434,G48079180,48,79180,2805826,4879180,Wildorado,Wildorado CDP,57,U1,N,N,G4210,S,6090285.0,890.0,35.209206,-102.209515,13205.348811,6.091175e+06


US Census geographic codes are [here](https://www.census.gov/programs-surveys/geography/technical-documentation/code-lists.html#:~:text=The%20class%20(CLASSFP)%20code%20defines,class%20of%20a%20geographic%20entity.&text=The%20legal%2Fstatistical%20area%20description,typology%20for%20each%20geographic%20entity.&text=The%20functional%20status%20(FUNCSTAT)%20code,status%20of%20a%20geographic%20entity.) and state FIPS codes are [here](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt)

All we need to scrape Wikipedia pages are the town name and state, so we'll make a shorter df with just 'NAME' and 'STATEFP' and convert the latter to the state name

In [65]:
places = places_qgis[['NAME', 'STATEFP']].copy()
places

,NAME,STATEFP
0,Ainsworth,31
1,Albion,31
2,Amherst,31
3,Anoka,31
4,Arcadia,31
...,...,...
431,Vega,48
432,Weinert,48
433,Wellington,48
434,Wildorado,48


In [66]:
places.dtypes

NAME       object
STATEFP     int64
dtype: object

In [67]:
places['STATEFP'].unique()

array([31, 38, 46, 20, 30, 40, 48])

In [68]:
state_codes = {
    '31': 'Nebraska',
    '38': 'North Dakota',
    '46': 'South Dakota',
    '20': 'Kansas',
    '30': 'Montana',
    '40': 'Oklahoma',
    '48': 'Texas'
}

places['State'] = places['STATEFP'].astype(str).map(state_codes)
places

,NAME,STATEFP,State
0,Ainsworth,31,Nebraska
1,Albion,31,Nebraska
2,Amherst,31,Nebraska
3,Anoka,31,Nebraska
4,Arcadia,31,Nebraska
...,...,...,...
431,Vega,48,Texas
432,Weinert,48,Texas
433,Wellington,48,Texas
434,Wildorado,48,Texas


In [69]:
places = places.drop(columns=['STATEFP']).rename(columns={'NAME': 'Name'})
places

,Name,State
0,Ainsworth,Nebraska
1,Albion,Nebraska
2,Amherst,Nebraska
3,Anoka,Nebraska
4,Arcadia,Nebraska
...,...,...
431,Vega,Texas
432,Weinert,Texas
433,Wellington,Texas
434,Wildorado,Texas


### Scraping population data from Wikipedia us-census-pop tables for each town

First I generate a wikipedia url for every town, assuming the url exists

In [70]:
base_url = "https://en.wikipedia.org/wiki/"

In [71]:
places['wiki_url'] = (
    base_url
    + places['Name'].str.replace(" ", "_")
    + ",_"
    + places['State'].str.replace(" ", "_")
)

places

,Name,State,wiki_url
0,Ainsworth,Nebraska,"https://en.wikipedia.org/wiki/Ainsworth,_Nebraska"
1,Albion,Nebraska,"https://en.wikipedia.org/wiki/Albion,_Nebraska"
2,Amherst,Nebraska,"https://en.wikipedia.org/wiki/Amherst,_Nebraska"
3,Anoka,Nebraska,"https://en.wikipedia.org/wiki/Anoka,_Nebraska"
4,Arcadia,Nebraska,"https://en.wikipedia.org/wiki/Arcadia,_Nebraska"
...,...,...,...
431,Vega,Texas,"https://en.wikipedia.org/wiki/Vega,_Texas"
432,Weinert,Texas,"https://en.wikipedia.org/wiki/Weinert,_Texas"
433,Wellington,Texas,"https://en.wikipedia.org/wiki/Wellington,_Texas"
434,Wildorado,Texas,"https://en.wikipedia.org/wiki/Wildorado,_Texas"


Scrape the list of wiki_urls and create a list of town dicts

I want this data structure:

```
towns = [
    {   
        'name': ...,
        'state': ...,
        'pop_data': [ 
            {
                'Year': ...,
                'Population': ...,
                'Notes': ...,
                'Percent change': ...
            },
            ...
        ]
    },
    ...
]
```

In [72]:
towns = [] # list of town dicts

invalid_urls = [] # towns with invalid wiki_url
no_pop_table = [] # towns with wiki_url but no us-census-pop table

for i, row in places.iterrows():
    name = row['Name']
    state = row['State']
    wiki_url = row['wiki_url']

    try:
        response = requests.get(wiki_url, timeout=10)
        response.raise_for_status() # in case wiki_url returns error

        doc = BeautifulSoup(response.text, "html.parser")
        pop_table = doc.find('table', class_='us-census-pop')

        if not pop_table:
            print(f"No pop table for {name}, {state}")
            no_pop_table.append({'Name': name, 'State': state, 'wiki_url': wiki_url})
            continue

        town_data = {
            'name': name,
            'state': state,
            'pop_data': []
        }

        rows = pop_table.find_all('tr')[1:] # skip header row

        for row in rows:
            cols = row.find_all(['th', 'td'])
            if len(cols) >= 2:
                pop_data = {
                    'year': cols[0].text.strip(),
                    'population': cols[1].text.strip(),
                    # 'notes': cols[2].text.strip() if len(cols) > 2 else "", # don't want this col
                    'percent_change': cols[3].text.strip() if len(cols) >= 3 else ""
                }
                town_data['pop_data'].append(pop_data)
    
        towns.append(town_data)

    except requests.exceptions.RequestException:
        print(f"Invalid wiki_url for {name}, {state}")
        invalid_urls.append({'Name': name, 'State': state, 'wiki_url': wiki_url})

towns

Invalid wiki_url for Howard City (Boelus), Nebraska
No pop table for Caputa, South Dakota
No pop table for Martinsdale, Montana
No pop table for Selman, Oklahoma
No pop table for Allison, Texas


[{'name': 'Ainsworth',
  'state': 'Nebraska',
  'pop_data': [{'year': '1890', 'population': '733', 'percent_change': '—'},
   {'year': '1900', 'population': '605', 'percent_change': '−17.5%'},
   {'year': '1910', 'population': '1,045', 'percent_change': '72.7%'},
   {'year': '1920', 'population': '1,508', 'percent_change': '44.3%'},
   {'year': '1930', 'population': '1,378', 'percent_change': '−8.6%'},
   {'year': '1940', 'population': '1,833', 'percent_change': '33.0%'},
   {'year': '1950', 'population': '2,150', 'percent_change': '17.3%'},
   {'year': '1960', 'population': '1,982', 'percent_change': '−7.8%'},
   {'year': '1970', 'population': '2,073', 'percent_change': '4.6%'},
   {'year': '1980', 'population': '2,256', 'percent_change': '8.8%'},
   {'year': '1990', 'population': '1,870', 'percent_change': '−17.1%'},
   {'year': '2000', 'population': '1,862', 'percent_change': '−0.4%'},
   {'year': '2010', 'population': '1,728', 'percent_change': '−7.2%'},
   {'year': '2020', 'popula

In [73]:
len(towns)

759

Saved the towns with `invalid_urls` or `no-pop-table` in separate lists of dicts so I can double-check them or find the populations elsewhere. In this case, there's only 1 with a non-existent wikipedia page and 4 without us-census-pop tables, and we can leave them out, bringing the total number of scraped towns to 759.

Note: The Allison, Texas wikipedia page doesn't have a us-census-pop table because there are apparently [3 towns that have been named Allison, Texas](https://en.wikipedia.org/wiki/Allison,_Texas), but none of the individual pages has a us-census-pop table either

In [74]:
invalid_urls

[{'Name': 'Howard City (Boelus)',
  'State': 'Nebraska',
  'wiki_url': 'https://en.wikipedia.org/wiki/Howard_City_(Boelus),_Nebraska'}]

In [75]:
no_pop_table

[{'Name': 'Caputa',
  'State': 'South Dakota',
  'wiki_url': 'https://en.wikipedia.org/wiki/Caputa,_South_Dakota'},
 {'Name': 'Martinsdale',
  'State': 'Montana',
  'wiki_url': 'https://en.wikipedia.org/wiki/Martinsdale,_Montana'},
 {'Name': 'Selman',
  'State': 'Oklahoma',
  'wiki_url': 'https://en.wikipedia.org/wiki/Selman,_Oklahoma'},
 {'Name': 'Allison',
  'State': 'Texas',
  'wiki_url': 'https://en.wikipedia.org/wiki/Allison,_Texas'}]

Now flatten the nested `towns` data structure so I can convert to DataFrame and then to csv

In [ ]:
# method 1: flatten manually with a for loop
# gives more control over column name and order

flat_rows = []

for town in towns:
    name = town['name']
    state = town['state']
    for entry in town['pop_data']:
        flat_row = {
            'Name': name,
            'State': state,
            'Year': entry['year'],
            'Population': entry['population'],
            'Percent change': entry['percent_change']
        }
        flat_rows.append(flat_row)

towns_df = pd.DataFrame(flat_rows)
towns_df

,Name,State,Year,Population,Percent change
0,Ainsworth,Nebraska,1890,733,—
1,Ainsworth,Nebraska,1900,605,−17.5%
2,Ainsworth,Nebraska,1910,"1,045",72.7%
3,Ainsworth,Nebraska,1920,"1,508",44.3%
4,Ainsworth,Nebraska,1930,"1,378",−8.6%
...,...,...,...,...,...
8626,Wilson,Texas,1980,578,33.5%
8627,Wilson,Texas,1990,568,−1.7%
8628,Wilson,Texas,2000,532,−6.3%
8629,Wilson,Texas,2010,489,−8.1%


### Cleaning scraped population data

A few things to clean:
- remove % sign from `Percent change`
- remove , from `Population` so can change to int
- change `Year`, `Population`, `Percent change` to int or float
- some years after 2020 are given as (est.), remove these

In [77]:
# drop rows with (est.) in Year
towns_df = towns_df[~towns_df['Year'].str.contains(r'\(est\.\)', na=False)].copy()

# clean formatting in string columns
towns_df['Percent change'] = (
    towns_df['Percent change']
    .str.replace('−', '-', regex=False)   # Unicode minus → ASCII minus
    .str.rstrip('%')                      # Remove percent sign
)

towns_df['Population'] = towns_df['Population'].str.replace(',', '', regex=False)

# convert columns to numeric
towns_df[['Year', 'Population', 'Percent change']] = towns_df[[
    'Year', 'Population', 'Percent change'
]].apply(pd.to_numeric, errors='coerce')

In [78]:
towns_df.dtypes

Name               object
State              object
Year                int64
Population          int64
Percent change    float64
dtype: object

In [79]:
towns_df.head(20)

,Name,State,Year,Population,Percent change
0,Ainsworth,Nebraska,1890,733,NaN
1,Ainsworth,Nebraska,1900,605,-17.5
2,Ainsworth,Nebraska,1910,1045,72.7
3,Ainsworth,Nebraska,1920,1508,44.3
4,Ainsworth,Nebraska,1930,1378,-8.6
5,Ainsworth,Nebraska,1940,1833,33.0
6,Ainsworth,Nebraska,1950,2150,17.3
7,Ainsworth,Nebraska,1960,1982,-7.8
8,Ainsworth,Nebraska,1970,2073,4.6
9,Ainsworth,Nebraska,1980,2256,8.8


In [80]:
# change 'Percent change' NaN (usually first census year recorded) to 0
towns_df['Percent change'] = towns_df['Percent change'].fillna(0)
towns_df.head()

,Name,State,Year,Population,Percent change
0,Ainsworth,Nebraska,1890,733,0.0
1,Ainsworth,Nebraska,1900,605,-17.5
2,Ainsworth,Nebraska,1910,1045,72.7
3,Ainsworth,Nebraska,1920,1508,44.3
4,Ainsworth,Nebraska,1930,1378,-8.6


Just a few things to check that everything looks OK

In [91]:
towns_df.shape

(8581, 5)

In [93]:
towns_df['State'].unique()

array(['Nebraska', 'North Dakota', 'South Dakota', 'Kansas', 'Montana',
       'Oklahoma', 'Texas'], dtype=object)

In [84]:
towns_df['Year'].max()

np.int64(2020)

In [85]:
towns_df['Year'].min()

np.int64(1860)

In [87]:
towns_df['Population'].max()

np.int64(12438)

`town_populations.csv` is all scraped data from 1860-2020

In [ ]:
towns_df.to_csv("data/town_populations.csv", index=False)

There are some towns that were founded / have population data only starting after the relevant railroad town-founding period, so let's remove these. Let's assume that if railroads began declining around 1920, but a town has data starting in 1930, it is plausible that it was founded as early as ~1920 but just no census data for it yet

In [2]:
towns_df = pd.read_csv("data/town_populations.csv")
towns_df.head()

,Name,State,Year,Population,Percent change
0,Ainsworth,Nebraska,1890,733,0.0
1,Ainsworth,Nebraska,1900,605,-17.5
2,Ainsworth,Nebraska,1910,1045,72.7
3,Ainsworth,Nebraska,1920,1508,44.3
4,Ainsworth,Nebraska,1930,1378,-8.6


In [12]:
towns_df.shape

(8581, 5)

In [21]:
# find earliest year for each town, state pair (since some towns in different states have the same name)
first_years = towns_df.groupby(['Name', 'State'])['Year'].min()

# keep only towns that start in 1920 or earlier
valid_towns = first_years[first_years <= 1930].index

# filter towns_df to valid_towns, first setting index to the town, state pair to aid selection, then resetting the index to put them back into columns
early_towns = towns_df.set_index(['Name', 'State']).loc[valid_towns].reset_index()
early_towns.head(10)

,Name,State,Year,Population,Percent change
0,Admire,Kansas,1920,205,0.0
1,Admire,Kansas,1930,263,28.3
2,Admire,Kansas,1940,270,2.7
3,Admire,Kansas,1950,184,-31.9
4,Admire,Kansas,1960,149,-19.0
5,Admire,Kansas,1970,144,-3.4
6,Admire,Kansas,1980,158,9.7
7,Admire,Kansas,1990,147,-7.0
8,Admire,Kansas,2000,177,20.4
9,Admire,Kansas,2010,156,-11.9


In [20]:
early_towns.shape

(8299, 5)

In [27]:
# double-check what I removed
invalid_towns = first_years[first_years > 1930].index

late_towns = towns_df.set_index(['Name', 'State']).loc[invalid_towns].reset_index()

late_towns.head(25)

,Name,State,Year,Population,Percent change
0,Ada,Kansas,2010,100,0.0
1,Ada,Kansas,2020,86,-14.0
2,Adams,Oklahoma,2020,148,0.0
3,Adrian,Texas,1960,258,0.0
4,Adrian,Texas,1970,228,-11.6
5,Adrian,Texas,1980,222,-2.6
6,Adrian,Texas,1990,220,-0.9
7,Adrian,Texas,2000,159,-27.7
8,Adrian,Texas,2010,166,4.4
9,Adrian,Texas,2020,128,-22.9


`early_town_populations.csv` is population data for towns that have data starting in 1930 or earlier

In [28]:
early_towns.to_csv("data/early_town_populations.csv", index=False)

I also want a csv of data filtered by state

In [29]:
states = early_towns['State'].unique()

In [30]:
states.shape

(7,)

In [32]:
for state in states:
    state_data = early_towns[early_towns['State'] == state]
    filename = Path ("data") / f"{state.lower().replace(' ','_')}_populations.csv"
    state_data.to_csv(filename, index=False)

For charting purposes, I also want the median percentage change (not mean, since these towns are so small that there can be huge swings in percent) for each decade across all towns in each state

In [35]:
early_towns.dtypes

Name               object
State              object
Year                int64
Population          int64
Percent change    float64
dtype: object

In [37]:
early_towns_medians_long = (early_towns.groupby(['State', 'Year'], as_index=False)['Percent change']).median().rename(columns={'Percent change': 'Median percent change'})

early_towns_medians_long

,State,Year,Median percent change
0,Kansas,1860,0.00
1,Kansas,1870,0.00
2,Kansas,1880,0.00
3,Kansas,1890,28.00
4,Kansas,1900,1.95
...,...,...,...
99,Texas,1980,1.70
100,Texas,1990,-14.10
101,Texas,2000,-6.80
102,Texas,2010,-7.10


In [ ]:
early_towns_medians_long = early_towns_medians_long.round(1) # round to one decimal point

In [40]:
early_towns_medians_long.to_csv("data/early_towns_medians_long.csv", index=False)

In [41]:
early_towns_medians_wide = early_towns_medians_long.pivot(index='Year', columns='State', values='Median percent change').sort_index()

early_towns_medians_wide

State,Kansas,Montana,Nebraska,North Dakota,Oklahoma,South Dakota,Texas
Year,,,,,,,
1860,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1870,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1880,0.0,NaN,0.0,0.0,NaN,0.0,0.0
1890,28.0,0.0,0.0,0.0,NaN,0.0,0.0
1900,2.0,NaN,0.0,21.8,0.0,6.5,NaN
1910,3.2,0.0,27.2,0.0,0.0,6.8,0.0
1920,0.0,0.0,14.1,3.6,0.0,11.7,0.0
1930,-2.0,-16.5,0.0,10.1,10.9,-3.2,31.6
1940,-4.2,3.8,-3.2,8.4,-9.5,-1.6,-1.5


In [43]:
early_towns_medians_wide.to_csv("data/early_towns_medians_wide.csv")